# Intro to F1 

Jupyter Notebook
- 'm' convert cell to markdown
- 'a' insert cell above
- 'dd' delete cell
- 'b' insrert cell below
- 'Shift + enter' runs the block and move to next one

In [ ]:
import fastf1 as ff1
from fastf1 import plotting

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.colors import ListedColormap
from matplotlib.collections import LineCollection

import numpy as np
import pandas as pd


In [ ]:

# Enable the cache by providing the name of the cache folder
ff1.Cache.enable_cache('../cache')

year, grand_prix, session = 2022, 'Zandvoort', 'Q'

session = ff1.get_session(year, grand_prix, session)

session.load()

In [ ]:
#Enable plotting settings
ff1.plotting.setup_mpl()

## 1. Explore the data

In [ ]:
laps_first_driver = session.laps.pick_driver('ALB')

In [ ]:
laps_first_driver.pick_fastest()

## 2. Building the plot

In [ ]:
driver_1, driver_2 = 'VET', 'NOR'

In [ ]:
laps_driver_1 = session.laps.pick_driver(driver_1)
laps_driver_2 = session.laps.pick_driver(driver_2)

fastest_driver_1 = laps_driver_1.pick_fastest()
fastest_driver_2 = laps_driver_2.pick_fastest()

telemetry_driver_1 = fastest_driver_1.get_telemetry()
telemetry_driver_2 = fastest_driver_2.get_telemetry()

In [ ]:
# Delta time
delta_time, ref_tel, compare_tel = ff1.utils.delta_time(fastest_driver_1, fastest_driver_2)

In [ ]:
# identify team color
team_driver_1 = laps_driver_1['Team'].iloc[0]
team_driver_2 = laps_driver_2['Team'].iloc[0]

color_1 = ff1.plotting.team_color(team_driver_1)
color_2 = ff1.plotting.team_color(team_driver_2)

## 2.1 Telemetry Plot

In [ ]:
plt.rcParams['figure.figsize'] = [20, 15]

# 7 plots
# - Delta
# - Speed
# - THrottle
# - Braking
# - Gear
# - RPM
# - DRS
fig, ax = plt.subplots(7, gridspec_kw={'height_ratios': [1,3,2,1,1,2,1]})

# Set the title of the plot. 
ax[0].title.set_text(f'Telemetry comparison {driver_1} vs. {driver_2}')

# Subplot 1: The Delta
ax[0].plot(ref_tel['Distance'], delta_time, color=color_1)
# We want to add a middle line, the 0 line
ax[0].axhline(0)
ax[0].set(ylabel=f'Gap to {driver_2} (s)')

# Subplot 2: The Delta
ax[1].plot(telemetry_driver_1['Distance'], telemetry_driver_1['Speed'],label=driver_1, color=color_1)
ax[1].plot(telemetry_driver_2['Distance'], telemetry_driver_2['Speed'], label=driver_2,color=color_2)

ax[1].set(ylabel='Speed')
ax[1].legend(loc='lower right')

# Subplot 3: Throttle
ax[2].plot(telemetry_driver_1['Distance'], telemetry_driver_1['Throttle'],label=driver_1, color=color_1)
ax[2].plot(telemetry_driver_2['Distance'], telemetry_driver_2['Throttle'], label=driver_2,color=color_2)

ax[2].set(ylabel='Throttle')

# Subplot 4: Braking
ax[3].plot(telemetry_driver_1['Distance'], telemetry_driver_1['Brake'],label=driver_1, color=color_1)
ax[3].plot(telemetry_driver_2['Distance'], telemetry_driver_2['Brake'], label=driver_2,color=color_2)

ax[3].set(ylabel='Braking')

# Subplot 5: Gear
ax[4].plot(telemetry_driver_1['Distance'], telemetry_driver_1['nGear'],label=driver_1, color=color_1)
ax[4].plot(telemetry_driver_2['Distance'], telemetry_driver_2['nGear'], label=driver_2,color=color_2)

ax[4].set(ylabel='Gear')

# Subplot 6: RPM
ax[5].plot(telemetry_driver_1['Distance'], telemetry_driver_1['RPM'],label=driver_1, color=color_1)
ax[5].plot(telemetry_driver_2['Distance'], telemetry_driver_2['RPM'], label=driver_2,color=color_2)

ax[5].set(ylabel='RPM')

# Subplot 6: DRS
ax[6].plot(telemetry_driver_1['Distance'], telemetry_driver_1['DRS'],label=driver_1, color=color_1)
ax[6].plot(telemetry_driver_2['Distance'], telemetry_driver_2['DRS'], label=driver_2,color=color_2)

ax[6].set(ylabel='DRS')

## 2.2 Minisector comparison

In [ ]:
#Merge the telemetry from both drivers into one dataframe
telemetry_driver_1['Driver'] = driver_1
telemetry_driver_2['Driver'] = driver_2

telemetry = pd.concat([telemetry_driver_1, telemetry_driver_2])

In [ ]:
# Calculate the minisector
num_minisectors = 25
total_distance = max(telemetry['Distance'])
minisector_length = total_distance / num_minisectors

In [ ]:
minisectors = [0]
for i in range(0, (num_minisectors -1 )):
    minisectors.append(minisector_length *(i + 1))

In [ ]:
# Assign a minisector number to every row in trhe telemetry dataframe
telemetry['Minisector'] = telemetry['Distance'].apply(
    lambda dist:(
        int((dist // minisector_length) + 1)
    )
)

In [ ]:
# Calculate minisector per driver
average_speed = telemetry.groupby(['Minisector', 'Driver'])['Speed'].mean().reset_index()


In [ ]:
fastest_driver = average_speed.loc[average_speed.groupby(['Minisector'])['Speed'].idxmax()]
fastest_driver = fastest_driver[['Minisector', 'Driver']].rename(columns={'Driver': 'Fastest_driver'})

In [ ]:
#merge the fastest_driver dataframe to the telemetry dataframe on minisector
telemetry = telemetry.merge(fastest_driver, on=['Minisector'])
telemetry = telemetry.sort_values(by=['Distance'])

In [ ]:
telemetry.loc[telemetry['Fastest_driver'] == driver_1, 'Fastest_driver_int'] = 1
telemetry.loc[telemetry['Fastest_driver'] == driver_2, 'Fastest_driver_int'] = 2


In [ ]:
x = np.array(telemetry['X'].values)
y = np.array(telemetry['Y'].values)



In [ ]:
#convert to points
points = np.array([x, y]).T.reshape(-1, 1, 2)

segments = np.concatenate([points[:-1], points[1:]], axis=1)
fastest_driver_array = telemetry['Fastest_driver_int'].to_numpy().astype(float)

fastest_driver_array

In [ ]:
# the segments we juste created can now be coloroed  according to the fasted drvier in a minisecotr
cmap = ListedColormap([color_1, color_2])
lc_comp = LineCollection(segments, norm=plt.Normalize(1, cmap.N+1), cmap=cmap)
lc_comp.set_array(fastest_driver_array)
lc_comp.set_linewidth(5)

In [ ]:
plt.rcParams['figure.figsize'] = [18, 10]

plt.gca().add_collection(lc_comp)
plt.axis('equal')
plt.box(False)
plt.tick_params(labelleft=False, left=False, labelbottom=False, bottom=False)

# add a color bar fopr the legend
cbar = plt.colorbar(mappable=lc_comp, boundaries=np.arange(1,4))
cbar.set_ticks(np.arange(1, 9))
cbar.set_ticklabels([driver_1, driver_2])